In [6]:
import os
import math
import pickle
import numpy as np
from idsse_helper import load_data
from numpy.lib.format import open_memmap


AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)

In [ ]:
# ----- Configuration -----
DATA_DIR    = '../data/idsse-data/data/'
OUTPUT_DIR_WHOLE  = '../data/idsse-data/seqlabelvae/whole'
OUTPUT_DIR_SPLIT  = '../data/idsse-data/seqlabelvae/split'
H, W        = 105, 68
WINDOW_SIZE = 32
FRAME_RATE  = 25
HALF_W      = WINDOW_SIZE // 2

FILE_GROUPS = [
     # (pos_xml, info_xml, events_xml),
     ("DFL_04_03_positions_raw_observed_DFL-COM-000001_DFL-MAT-J03WMX.xml","DFL_02_01_matchinformation_DFL-COM-000001_DFL-MAT-J03WMX.xml","DFL_03_02_events_raw_DFL-COM-000001_DFL-MAT-J03WMX.xml"),
     ("DFL_04_03_positions_raw_observed_DFL-COM-000001_DFL-MAT-J03WN1.xml","DFL_02_01_matchinformation_DFL-COM-000001_DFL-MAT-J03WN1.xml","DFL_03_02_events_raw_DFL-COM-000001_DFL-MAT-J03WN1.xml"),
     ("DFL_04_03_positions_raw_observed_DFL-COM-000002_DFL-MAT-J03WOH.xml","DFL_02_01_matchinformation_DFL-COM-000002_DFL-MAT-J03WOH.xml","DFL_03_02_events_raw_DFL-COM-000002_DFL-MAT-J03WOH.xml"),
     ("DFL_04_03_positions_raw_observed_DFL-COM-000002_DFL-MAT-J03WOY.xml","DFL_02_01_matchinformation_DFL-COM-000002_DFL-MAT-J03WOY.xml","DFL_03_02_events_raw_DFL-COM-000002_DFL-MAT-J03WOY.xml"),
     ("DFL_04_03_positions_raw_observed_DFL-COM-000002_DFL-MAT-J03WPY.xml","DFL_02_01_matchinformation_DFL-COM-000002_DFL-MAT-J03WPY.xml","DFL_03_02_events_raw_DFL-COM-000002_DFL-MAT-J03WPY.xml"),
     ("DFL_04_03_positions_raw_observed_DFL-COM-000002_DFL-MAT-J03WQQ.xml","DFL_02_01_matchinformation_DFL-COM-000002_DFL-MAT-J03WQQ.xml","DFL_03_02_events_raw_DFL-COM-000002_DFL-MAT-J03WQQ.xml"),
     ("DFL_04_03_positions_raw_observed_DFL-COM-000002_DFL-MAT-J03WR9.xml","DFL_02_01_matchinformation_DFL-COM-000002_DFL-MAT-J03WR9.xml","DFL_03_02_events_raw_DFL-COM-000002_DFL-MAT-J03WR9.xml"),
   ]


# Load event-to-index map
with open(os.path.join('../data/idsse-data/events_labels.pkl'), 'rb') as f:
    EVENTS_LABELS = pickle.load(f)

: 

In [ ]:
def round_and_clip(x, y):
    xg = int(np.round(x)); yg = int(np.round(y))
    return np.clip(xg, 0, H-1), np.clip(yg, 0, W-1)

def pad_positions(arr, target=22):
    vals = arr[~np.isnan(arr)]
    vals = vals[:target*2].reshape(-1,2)
    return vals

: 

In [ ]:
# 1) First pass: compute total frames
frames_per_match = []
for fpos, finfo, fevents in FILE_GROUPS:
    xy, ev, _ = load_data(DATA_DIR, fpos, finfo, fevents)
    T1 = xy['firstHalf']['Home'].xy.shape[0]
    T2 = xy['secondHalf']['Home'].xy.shape[0]
    frames_per_match.append(T1 + T2)
total_frames = sum(frames_per_match)

: 